<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.19.5
Pandas:  1.1.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [5]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

                                                   0
0  instant,dteday,season,yr,mnth,hr,holiday,weekd...
1  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...
2  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...
3  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...
4  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...

  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
0  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...     
1  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...     
2  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...     
3  5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,...     
4  6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,...     

  instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...                                                                   
1  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...                                        

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [6]:
bikes = pd.read_csv('bikeshare.csv')
bikes.head()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [7]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [8]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [9]:
df.reset_index(drop=True)


,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [10]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [11]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df = pd.read_html(url)
df

[                         Bank NameBank  ... FundFund
 0                    Almena State Bank  ...    10538
 1           First City Bank of Florida  ...    10537
 2                 The First State Bank  ...    10536
 3                   Ericson State Bank  ...    10535
 4     City National Bank of New Jersey  ...    10534
 ..                                 ...  ...      ...
 558                 Superior Bank, FSB  ...     6004
 559                Malta National Bank  ...     4648
 560    First Alliance Bank & Trust Co.  ...     4647
 561  National State Bank of Metropolis  ...     4646
 562                   Bank of Honolulu  ...     4645
 
 [563 rows x 7 columns]]

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [12]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [13]:
bikes.shape


(17379, 17)

What are the column names in `bikes`?

In [14]:
bikes.columns


Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [15]:
bikes.dtypes


instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [16]:
row=bikes.iloc[0]
print(row)




instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed              0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object


https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [17]:
bikes['season']




0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

In [18]:
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [19]:
atemps = bikes.atemp
atemps[0:4]


0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [20]:
#bikes[[0]]
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: ignored

What is the correct way to access the 1st row of the DataFrame by its index?

In [21]:
bikes.iloc[0]



instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed              0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [22]:
bikes.iloc[:,1]


0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [23]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [24]:
bikes.isnull().sum()


instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [25]:
bikes.isnull()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17375,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17376,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17377,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [25]:
result = float('') * 10


ValueError: ignored

Apply the Pandas `isna` function to the following data objects:

In [26]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [27]:
pd.isna(x)



False

In [28]:
pd.isna(y)

True

In [29]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [30]:
pd.isna(array)


array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [31]:
bikes.head()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [32]:
select = bikes[['instant','dteday','windspeed']]
select

,instant,dteday,windspeed
0,1,2011-01-01,0.0000
1,2,2011-01-01,0.0000
2,3,2011-01-01,0.0000
3,4,2011-01-01,0.0000
4,5,2011-01-01,0.0000
...,...,...,...
17374,17375,2012-12-31,0.1642
17375,17376,2012-12-31,0.1642
17376,17377,2012-12-31,0.1642
17377,17378,2012-12-31,0.1343


In [33]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


Compute and print the overall minimum and maximum of the numeric data columns:

In [34]:
bikes_min, bikes_max = (bikes.min()), (bikes.max())
bikes_min, bikes_max

(instant                1
 dteday        2011-01-01
 season                 1
 yr                     0
 mnth                   1
 hr                     0
 holiday                0
 weekday                0
 workingday             0
 weathersit             1
 temp                0.02
 atemp                  0
 hum                    0
 windspeed              0
 casual                 0
 registered             0
 cnt                    1
 dtype: object, instant            17379
 dteday        2012-12-31
 season                 4
 yr                     1
 mnth                  12
 hr                    23
 holiday                1
 weekday                6
 workingday             1
 weathersit             4
 temp                   1
 atemp                  1
 hum                    1
 windspeed         0.8507
 casual               367
 registered           886
 cnt                  977
 dtype: object)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [35]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [36]:
bikes['atemp'].quantile([0.1,0.25,0.5,0.75,0.9])

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [37]:
type(bikes['atemp'])

pandas.core.series.Series

In [38]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
4821,4822,2011-07-24,3,0,7,19,0,0,0,1,0.86,0.8030,0.47,0.1642,70,170,240
12136,12137,2012-05-25,2,1,5,21,0,5,1,1,0.70,0.6515,0.65,0.1343,62,193,255
10711,10712,2012-03-27,2,1,3,10,0,2,1,1,0.26,0.2576,0.35,0.2239,20,118,138
13911,13912,2012-08-07,3,1,8,20,0,2,1,2,0.74,0.6970,0.70,0.1343,74,374,448
17027,17028,2012-12-17,4,1,12,6,0,1,1,2,0.36,0.3485,0.93,0.1343,1,107,108


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [39]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [40]:
atemp_level


0        (0.25, 0.5]
1        (0.25, 0.5]
2        (0.25, 0.5]
3        (0.25, 0.5]
4        (0.25, 0.5]
            ...     
17374    (0.25, 0.5]
17375    (0.25, 0.5]
17376    (0.25, 0.5]
17377    (0.25, 0.5]
17378    (0.25, 0.5]
Name: atemp, Length: 17379, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

Here is a random sample of `atemp_level`:

In [41]:
atemp_level.sample(5)          

4024     (0.5, 0.75]
11924    (0.5, 0.75]
3748     (0.5, 0.75]
5735     (0.5, 0.75]
5346     (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [42]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

9506     cool
8341     cool
8826     mild
11454    mild
12783     hot
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [99]:
atemp_level
bikes['atemp_level'] = atemp_level


In [98]:
sea2 = bikes.groupby(["season"]).get_group(2)
sea2

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
1807,1808,2011-03-21,2,0,3,0,0,1,1,3,0.34,0.3030,0.66,0.3881,2,11,13,mild
1808,1809,2011-03-21,2,0,3,1,0,1,1,2,0.34,0.3030,0.71,0.3881,1,6,7,mild
1809,1810,2011-03-21,2,0,3,2,0,1,1,2,0.34,0.3030,0.71,0.3284,1,5,6,mild
1810,1811,2011-03-21,2,0,3,3,0,1,1,2,0.34,0.3030,0.71,0.3284,0,1,1,mild
1811,1812,2011-03-21,2,0,3,5,0,1,1,1,0.32,0.3030,0.81,0.2985,1,1,2,mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12758,12759,2012-06-20,2,1,6,19,0,3,1,1,0.88,0.7879,0.37,0.2537,90,449,539,hot
12759,12760,2012-06-20,2,1,6,20,0,3,1,1,0.86,0.7727,0.39,0.2239,81,326,407,hot
12760,12761,2012-06-20,2,1,6,21,0,3,1,1,0.84,0.7576,0.44,0.0896,42,302,344,hot
12761,12762,2012-06-20,2,1,6,22,0,3,1,1,0.82,0.7576,0.46,0.1045,31,226,257,hot


In [97]:
sea2.atemp_level.value_counts().mild

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [95]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [96]:
a.shape, b.shape, c.shape, d.shape, e.shape

((3,), (1, 3), (2, 3, 4), (4,), (2, 4))

## Load Data

Load rock.csv and clean the dataset.

In [17]:
rocks = pd.read_csv('rock.csv')
rocks

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1.0,0.0,3.0,0.0
2,Hold On Loosely,.38 Special,1981.0,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
3,Rockin' Into the Night,.38 Special,1980.0,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
844710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df=rocks.dropna()
df



,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
2,Hold On Loosely,.38 Special,1981.0,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
3,Rockin' Into the Night,.38 Special,1980.0,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0
5,Kryptonite,3 Doors Down,2000.0,Kryptonite by 3 Doors Down,1.0,1.0,13.0,13.0
...,...,...,...,...,...,...,...,...
2224,Sharp Dressed Man,ZZ Top,1983.0,Sharp Dressed Man by ZZ Top,1.0,1.0,120.0,120.0
2226,Tube Snake Boogie,ZZ Top,1981.0,Tube Snake Boogie by ZZ Top,1.0,1.0,32.0,32.0
2227,Tush,ZZ Top,1975.0,Tush by ZZ Top,1.0,1.0,109.0,109.0
2228,TV Dinners,ZZ Top,1983.0,TV Dinners by ZZ Top,1.0,1.0,1.0,1.0


## Check Column Names

Check column names and clean.

In [19]:
df.dropna(axis=1)

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
2,Hold On Loosely,.38 Special,1981.0,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
3,Rockin' Into the Night,.38 Special,1980.0,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0
5,Kryptonite,3 Doors Down,2000.0,Kryptonite by 3 Doors Down,1.0,1.0,13.0,13.0
...,...,...,...,...,...,...,...,...
2224,Sharp Dressed Man,ZZ Top,1983.0,Sharp Dressed Man by ZZ Top,1.0,1.0,120.0,120.0
2226,Tube Snake Boogie,ZZ Top,1981.0,Tube Snake Boogie by ZZ Top,1.0,1.0,32.0,32.0
2227,Tush,ZZ Top,1975.0,Tush by ZZ Top,1.0,1.0,109.0,109.0
2228,TV Dinners,ZZ Top,1983.0,TV Dinners by ZZ Top,1.0,1.0,1.0,1.0


## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [20]:
rocks.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1.0,0.0,3.0,0.0
2,Hold On Loosely,.38 Special,1981.0,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
3,Rockin' Into the Night,.38 Special,1980.0,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0


In [21]:
rocks["Release Year"].fillna("0", inplace = True)
rocks

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1.0,0.0,3.0,0.0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
844710,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
844711,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
844712,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
844713,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [22]:
rocks.dtypes

Song Clean       object
ARTIST CLEAN     object
Release Year     object
COMBINED         object
First?          float64
Year?           float64
PlayCount       float64
F*G             float64
dtype: object

In [23]:
rocks.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1.0,0.0,3.0,0.0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0


In [24]:
rocks['Release Year'] = rocks['Release Year'].astype('float').astype('int')
rocks.dtypes

Song Clean       object
ARTIST CLEAN     object
Release Year      int64
COMBINED         object
First?          float64
Year?           float64
PlayCount       float64
F*G             float64
dtype: object

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [11]:
rocks[['Release Year', 'First?', 'Year?','PlayCount', 'F*G']].max()

Release Year    2014.0
First?             1.0
Year?              1.0
PlayCount        142.0
F*G              142.0
dtype: float64

In [12]:
rocks[['Release Year', 'First?', 'Year?','PlayCount', 'F*G']].min()

Release Year    0.0
First?          1.0
Year?           0.0
PlayCount       0.0
F*G             0.0
dtype: float64

## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [13]:
rocks.index = np.arange(1,len(rocks)+1)
rocks.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
1,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
2,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1.0,0.0,3.0,0.0
3,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
4,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
5,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0


In [14]:
def song_details(row_num):
   song = rocks.loc[row_num, 'Song Clean']
   artist = rocks.loc[row_num, 'ARTIST CLEAN']
   year = rocks.loc[row_num, 'Release Year']
   if(year >= 1970):
    return song, artist, year
   else:
     return 'The year is less then 1970'
 
song_details(3)

('Hold On Loosely', '.38 Special', 1981)

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [16]:
rocks.dtypes

Song Clean       object
ARTIST CLEAN     object
Release Year      int64
COMBINED         object
First?          float64
Year?           float64
PlayCount       float64
F*G             float64
dtype: object

In [15]:
rocks.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
1,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1.0,1.0,82.0,82.0
2,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1.0,0.0,3.0,0.0
3,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1.0,1.0,85.0,85.0
4,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1.0,1.0,18.0,18.0
5,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1.0,1.0,1.0,1.0


In [31]:
def df(convert):
  first_col = rocks.loc[convert, 'First?']
  year_col = rocks.loc[convert, 'Year?']
  play_col = rocks.loc[convert, 'PlayCount']
  f_col = rocks.loc[convert, 'F*G']
  for first in first_col.select_dtypes(['object']):
    first = pd.to_numeric(first_col, errors='coerce')
    return first

df(5)



AttributeError: ignored

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

In [32]:
rocks.describe

<bound method NDFrame.describe of                     Song Clean ARTIST CLEAN  ...  PlayCount   F*G
0             Caught Up in You  .38 Special  ...       82.0  82.0
1                 Fantasy Girl  .38 Special  ...        3.0   0.0
2              Hold On Loosely  .38 Special  ...       85.0  85.0
3       Rockin' Into the Night  .38 Special  ...       18.0  18.0
4            Art For Arts Sake         10cc  ...        1.0   1.0
...                        ...          ...  ...        ...   ...
844710                     NaN          NaN  ...        NaN   NaN
844711                     NaN          NaN  ...        NaN   NaN
844712                     NaN          NaN  ...        NaN   NaN
844713                     NaN          NaN  ...        NaN   NaN
844714                     NaN          NaN  ...        NaN   NaN

[844715 rows x 8 columns]>

>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



